# Deep Learning Assignment 2023: Visual Grounding

The goal of the project is to develop a deep learning framework to perform visual grounding on the RefCOCOg dataset.

Throughout the work multiple approaches have been tested, yielding some very different results and with import differences in terms of performance. For the sake of the providing a complete overview of the work done, the notebook will go through all of the approach tested, presenting any idea that has been discussed and implemented in these months of work, discussing their pros and limitations. The approaches that have been taken into account include:

- a baseline pipeline using YOLO for object proposals and CLIP for the grounding task;
- a pipeline which involves image segmentation, CLIP embedding and bounding box proposal;
- a pipeline using DETR for object proposals and CLIP for the grounding task;
- a pipeline only using MDETR for the grounding task (for the sake of having a SOTA comparison)
- a framework involving Diffusion for bounding box detection and CLIP for the grounding task.
- a framework involving Reinforcement Learning for bounding box regression.

The notebook will cover all aforementioned approaches,

- (1) starting from a brief introduction, covering (i) the problem of visual grounding, (ii) a quick presentation of the dataset and (iii) CLIP
- (2) continuing through the implementation of the different frameworks, discussing pro and cons
- (3) and finally presenting the overall results achieved and providing some final considerations.

The notebook is meant to be run standalone here on Google Colab but, may it be useful, the codebase is also available on GitHub at [synchroazel/visual-grounding](https://github.com/synchroazel/visual-grounding). Please refer to the README for further instructions on how to run it locally.

## 1. Brief introduction



### 1.1 Visual Grounding

Visual grounding refers to the process of linking visual information with corresponding linguistic symbols, bridging the gap between visual perception and linguistic understanding. The goal is to enable machines to comprehend and interpret visual information in a way that is similar to human understanding.

Concretely, visual grounding can be approached in different ways, depending on the specific task and context. One common approach is to use techniques like object detection, image segmentation, or scene understanding to extract relevant visual features from an image or video. These features are then matched or aligned with corresponding textual descriptions or concepts using a multitude of approaches.

Many of the current approaches to visual grounding can be identified either as a **one-stage** or **two-stage**.

- Two-stage methods formulate visual grounding as a matching problem between language and region. The visual region proposals are extracted by a pre-trained detector in the first stage, which are matched with the given expression in the second stage. However, the performance of these methods is highly dependent on the detector in the first stage. Besides, matching must be performed for every region proposals, which drag a great extent on the speed of the network.

- One-stage methods overcome the reliance on detectors and speed up the inference process by grounding the object in an image by a sentence query directly.



### 1.2 RefCOCOg Dataset



### 1.3 Contrastive Language-Image Pre-Training (CLIP)



## Preliminary steps

Here are some preliminary steps before we start discussing each approach in more detail.

In [ ]:
#@title Import all necessary modules

import copy
import json
import os
import pickle
import re
import time
from datetime import datetime

import clip
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision.transforms as T
from PIL import Image
from skimage import io
from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.measure import regionprops
from skimage.segmentation import slic, watershed
from skimage.util import img_as_float
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset
from torchmultimodal.models.mdetr.model import mdetr_for_phrase_grounding
from torchvision import transforms
from torchvision.ops import box_iou
from torchvision.ops.boxes import box_convert
from tqdm import tqdm
from transformers import RobertaTokenizerFast
from ultralytics import YOLO

from modules.pipelines.clipssd import ClipSSD
from modules.pipelines.detrclip import DetrClip
from modules.pipelines.vgpipeline import VisualGroundingPipeline
from modules.utilities import cosine_similarity
from modules.utilities import display_preds, downsample_map


%matplotlib inline


In [ ]:
#@title Set the best device for to the machine running the notebook

def get_best_device():
    if torch.cuda.is_available():
        device_ = torch.device("cuda")  # for CUDA GPU
        print("[INFO] Using cuda.")
    elif torch.has_mps:
        device_ = torch.device("mps")  # for Apple Silicon GPU
        print("[INFO] Using MPS.")
    else:
        device_ = torch.device("cpu")
        print("[INFO] No GPU found, using CPU instead.")

    return device_


device = get_best_device()


It is also important to define our dataset and sample objects, which are going to be used throughout the notebook.

- The `RefCOCOgSample` object define a single sample from the RefCOCOg dataset, containing all the information - as attributes - that may be needed for visual grounding tasks.

- The `RefCOCOg` object is a wrapper around the RefCOCOg dataset, providing some useful methods to access the dataset and its samples. Most notably, it implements (1) a `__getitem__` method to access a single sample and (2) a `__len__` method to get the number of samples in the dataset. Also notice that the `__getitem__` method returns a dictionary containing the information that can be used to populate a `RefCOCOgSample` object (simply with `RefCOCOgSample(**output)`). The `__init__` method also takes care of loading the correct split we might need, accepting either `train`, `val` or `test` as arguments and using the annotations from `refs(umd).p` to select the samples of each.

In [ ]:
#@title Class definition for a sample from the RefCOCOg dataset

class RefCOCOgSample:
    """
    An annotated image from RefCOCOg dataset.

    """

    def __init__(self,
                 img: Image.Image,
                 shape: tuple[int, int],
                 path: str,
                 img_id: str,
                 split: str,
                 category: str,
                 category_id: int,
                 sentences: list[str],
                 bbox: list[float],
                 segmentation: list[float]):
        self.img = img
        self.shape = shape
        self.path = path
        self.id = img_id
        self.split = split
        self.category = category
        self.category_id = category_id
        self.sentences = sentences
        self.bbox = bbox
        self.segmentation = segmentation

    def __repr__(self):
        return str(vars(self))


In [ ]:
#@title Class definition for the RefCOCOg dataset

class RefCOCOg(Dataset):
    """
    Dataset object for RefCOCOg dataset.

    """

    def __init__(self, ds_path: str, split=None, transform=None):
        super(RefCOCOg, self).__init__()

        self.transform = transform

        self.ds_path = ds_path

        with open(f'{ds_path}/annotations/refs(umd).p', 'rb') as f:
            self.refs = pickle.load(f)

        with open(f"{ds_path}/annotations/instances.json", "r") as f:
            self.instances = json.load(f)

        self.categories = {
            item["id"]: {
                "supercategory": item["supercategory"],
                "category": item["name"]
            }
            for item in self.instances['categories']
        }

        if split == 'train':
            self.refs = [ref for ref in self.refs if ref['split'] == 'train']
        elif split == 'val':
            self.refs = [ref for ref in self.refs if ref['split'] == 'val']
        elif split == 'test':
            self.refs = [ref for ref in self.refs if ref['split'] == 'test']

        self.size = len(self.refs)

    def __getitem__(self, idx: int):

        refs_data = self.refs[idx]

        for inst in self.instances['annotations']:
            if inst['id'] == refs_data['ann_id']:
                ann_data = inst
                break

        image_path = os.path.join(
            self.ds_path,
            "images",
            re.sub(r"_[0-9]+\.jpg", ".jpg", refs_data["file_name"])
        )

        pil_img = Image.open(image_path)

        bbox = torch.tensor(ann_data["bbox"])
        bbox = box_convert(bbox, "xywh", "xyxy").numpy()

        sample = {
            "img": pil_img,
            "shape": transforms.ToTensor()(pil_img).shape,
            "path": image_path,
            "img_id": refs_data["image_id"],
            "split": refs_data["split"],
            "category": self.categories[refs_data["category_id"]]["category"],
            "category_id": refs_data["category_id"],
            "sentences": [sentence["raw"].lower() for sentence in refs_data["sentences"]],
            "bbox": bbox,
            "segmentation": ann_data["segmentation"]
        }

        if self.transform:
            sample = self.transform(sample["img"], dtype=torch.float32)

        return sample

    def __len__(self):
        return self.size


With that taken care of, we can now import the dataset and its split.

In [ ]:
#@title Import the RefCOCOg dataset and create train/validation/test splits

data_path = "dataset/refcocog"

dataset = RefCOCOg(ds_path=data_path)

train_ds = RefCOCOg(ds_path=data_path, split='train')
val_ds = RefCOCOg(ds_path=data_path, split='val')
test_ds = RefCOCOg(ds_path=data_path, split='test')

print(f"[INFO] Dataset Size: {len(dataset)}")
print(f"[INFO] train split:  {len(train_ds)}")
print(f"[INFO] val split:    {len(val_ds)}")
print(f"[INFO] test split:   {len(test_ds)}")


## 2. The pipelines

In the following section we are going to present the different visual grounding pipelines that have been implemented and tested.

**About the metrics**

Before diving into each approach, it is worth mentioning the metrics that have been used to evaluate the performance of each pipeline. The metrics used aim to evaluate the performance of the framework in terms of

(1) localization accuracy
(2) grounding accuracy
(3) semantic similarity

Localization accuracy refers to the ability of the model to localize an object in the image, and it is measured using **Intersection over Union (IoU)**, namely the ratio between the area of overlap between the predicted bounding box and the ground-truth bounding box and the area of union between the two.

Semantic similarity refers to the similarity between the predicted bounding boxes and the ground-truth descriptions, and it is measured using distance metrics such as cosine similarity, **euclidean distance** or **dot product** between the CLIP embeddings of the predicted bounding boxes and the ground-truth textual description.

Grounding accuracy refers to the ability of the model to ground the localized object to a language description, and it is measured using **recall**, namely the ratio between the number of correctly grounded objects and the total number of objects. Practically

- we get a CLIP encoding of each available category in a dummy `f"a picture of a {object}"` sentence
- we get a CLIP encoding of the bounding boxed image proposed by the pipeline
- we compare those and get the category that is most similar
- if the category is the same as the one in the ground truth, we have a correct grounding attempt


**About the implementation**

To simplify our codebase, and to provide extra readability, we used a common superclass `VisualGroundingPipeline` for all the visual grounding frameworks approached. This superclass provides a common interface for all the pipelines, such as

- (1) initialization of common attributes
- (2) a pair of methods to encode text and images using CLIP after taking care of tokenization/preprocess
- (3) a method to compute IoU
- (4) a method to compute the grounding accuracy and finally a method used to embed, on pipeline instantiation, all the available categories (useful, as mentioned, for the visual grounding accuracy computation).

All pipelines inherit from this superclass, and implement one or more custom methods, most notably one of them being a `__call__` method which contains the core logic and returns the metrics of interest.

At subclass level each pipeline also implement a set of common attributes to be used in the `__call__` method for displaying and testing purposes, namely

- `show=True` will display the image with the predicted bounding box
- `timeit=True` will print the time taken to run the pipeline (it is recommended to use it without any visualization)
-

In [ ]:
#@title Class definition for the VisualGroundingPipeline superclass

class VisualGroundingPipeline:

    def __init__(self,
                 categories,
                 clip_ver="RN50",
                 device="cpu",
                 quiet=True):
        self.categories = copy.deepcopy(categories)
        self.clip_ver = clip_ver
        self.clip_model, self.clip_prep = clip.load(clip_ver, device="cpu")
        self.device = device
        self.quiet = quiet

        # model is loaded to cpu first, and eventually moved to gpu
        # (trick Mac M1 to use f16 tensors)
        if self.device != "cpu":
            self.clip_model = self.clip_model.to(self.device)

        self._embed_categories()

    def _encode_text(self, text):
        text_ = clip.tokenize(text).to(self.device)

        with torch.no_grad():
            return self.clip_model.encode_text(text_)

    def _encode_img(self, image):
        image_ = self.clip_prep(image).unsqueeze(0).to(self.device)

        with torch.no_grad():
            return self.clip_model.encode_image(image_)

    @staticmethod
    def _IoU(pred_bbox, gt_bbox):
        iou = box_iou(
            torch.tensor(pred_bbox).unsqueeze(0),
            torch.tensor(gt_bbox).unsqueeze(0)
        ).item()

        return iou

    def _grounding_accuracy(self, img_sample, pred_image_enc):
        all_c_sims = dict()

        for category_id in self.categories.keys():
            cur_categ = self.categories[category_id]['category']
            cur_categ_enc = self.categories[category_id]['encoding'].float()

            all_c_sims[cur_categ] = cosine_similarity(pred_image_enc, cur_categ_enc)

        pred_category = max(all_c_sims, key=all_c_sims.get)

        # if not self.quiet:
        #     print(f"[INFO] true: {img_sample.category} | predicted: {pred_category}")

        return 1 if pred_category == img_sample.category else 0

    def _embed_categories(self):
        for category_id in self.categories.keys():
            cur_category = self.categories[category_id]['category']
            with torch.no_grad():
                cur_category_enc = self._encode_text(f"a photo of {cur_category}")
            self.categories[category_id].update({"encoding": cur_category_enc})

    def __call__(self, *args, **kwargs):
        return None

Also, before skipping ahead, we pick a random a sample image from the dataset to use as a quick test for the following sections.

In [ ]:
#@title Pick a random sample from the dataset

idx = np.random.randint(0, len(dataset))
sample = RefCOCOgSample(**dataset[idx])

plt.imshow(sample.img)
plt.axis("off")
plt.title(f"Sample #{idx}", loc="left")
plt.show()

for i, sentence in enumerate(sample.sentences):
    print(f"[INFO] Sentence #{i}: {sentence}")


### 2.1 YOLO + CLIP

The first approach that has been tested is a baseline pipeline using YOLO for object proposals and CLIP for the grounding task. The idea is to use YOLO to extract the bounding boxes of the objects in an image and then use CLIP to find the best match between the bounding boxes and the sentence query. This latter part is simply done by computing CLIP embeddings of the cropped bounding boxes and the sentence query and then computing the cosine similarity between the two embeddings. The bounding box with the highest similarity score is then selected as the best match.

**YOLO**, which funnily stands for "You Only Look Once," is a popular object detection model in computer vision. It revolutionized real-time object detection by proposing a unified framework that simultaneously predicts object bounding boxes and class probabilities in a single pass. YOLO divides the input image into a grid and applies convolutional neural networks to each grid cell to predict bounding boxes and class probabilities. This approach allows YOLO to achieve impressive detection speeds while maintaining competitive accuracy.

This approach has been tested both using different version of YOLO (YOLOv8x and YOLOv5su - which recently replaced YOLOv5s) and different visual backbones for CLIP (ResNet50, ResNet101 and ViT-L/14). The results are shown in the following table.

In [ ]:
#@title Class definition for the YOLO+Clip pipeline

class YoloClip(VisualGroundingPipeline):

    def __init__(self,
                 categories,
                 yolo_ver="yolov8x",
                 clip_ver="RN50",
                 device="cpu",
                 quiet=True):

        VisualGroundingPipeline.__init__(self, categories, clip_ver, device, quiet)

        self.yolo_ver = yolo_ver
        self.yolo_model = YOLO(self.yolo_ver + ".pt")

        valid_yolo_versions = ["yolov8x", "yolov5su"]
        if yolo_ver not in valid_yolo_versions:
            raise ValueError(f"Invalid YOLO version '{yolo_ver}'. Must be one of {valid_yolo_versions}.")

        print("[INFO] Initializing YoloClip pipeline")
        print(f"[INFO] YOLO version: {yolo_ver}")
        print("")

    def __call__(self, img_sample, prompt, show=False, show_yolo=False, timeit=False):

        if timeit:
            start = time.time()

        # Get sample image
        img = img_sample.img

        # Use YOLO to propose relevant objects
        yolo_results_ = self.yolo_model(img_sample.path, verbose=False)[0]
        yolo_results = yolo_results_.boxes.xyxy
        if not self.quiet:
            print(f"[INFO] YOLO found {yolo_results.shape[0]} objects")
        if yolo_results.shape[0] == 0:
            print(f"[WARN] YOLO ({self.yolo_ver}) couldn't find any object in {img_sample.path}!")
            return {"IoU": 0, "cosine": np.nan, "euclidean": np.nan, "dotproduct": np.nan, "grounding": np.nan}

        # Use CLIP to encode each relevant object image
        images_encs = list()
        for i in range(yolo_results.shape[0]):
            bbox = yolo_results[i, 0:4].cpu().numpy()
            sub_img = img.crop(bbox)
            with torch.no_grad():
                sub_img_enc = self._encode_img(sub_img)
            images_encs.append(sub_img_enc)
        images_encs = torch.cat(images_encs, dim=0)

        # Use CLIP to encode the text prompt
        prompt_enc = self._encode_text(prompt)

        # Compute the best bbox according to cosine similarity
        c_sims = cosine_similarity(prompt_enc, images_encs).squeeze()
        best_idx = int(c_sims.argmax())

        # Get best bbox
        pred_bbox = yolo_results[best_idx, 0:4].tolist()

        # Crop around the best bbox and encode
        pred_image = img.crop(pred_bbox)
        pred_image_enc = self._encode_img(pred_image)

        # Get ground truth bbox
        gt_bbox = img_sample.bbox

        """ Metrics computation """

        # Compute IoU
        iou = self._IoU(pred_bbox, gt_bbox)

        # Compute grounding accuracy
        grd_correct = self._grounding_accuracy(img_sample, pred_image_enc)

        # Compute distance metrics
        dotproduct = prompt_enc @ pred_image_enc.T  # dot product
        cosine_sim = cosine_similarity(prompt_enc, pred_image_enc)  # cosine similarity
        euclidean_dist = torch.cdist(prompt_enc, pred_image_enc, p=2).squeeze()  # euclidean distance

        """ Display results """

        # Show objects found by YOLO, if requested
        if show_yolo:
            plt.imshow(yolo_results_.plot())
            plt.axis("off")
            plt.title("YOLO findings")

        # Show the final prediction, if requested
        if show:
            display_preds(img, prompt, pred_bbox, gt_bbox, model_name=f"{self.yolo_ver} + CLIP({self.clip_ver})")

        # Print execution time, if requested
        if timeit:
            end = time.time()
            print(f"[INFO] Time elapsed: {end - start:.2f}s")

        return {
            "IoU": float(iou),
            "cosine": float(cosine_sim),
            "euclidean": float(euclidean_dist),
            "dotproduct": float(dotproduct),
            "grounding": float(grd_correct),
        }


With that said, we can instantiate two object for the YOLO+CLIP pipeline, using different YOLO versions, and quickly see them running.

In [ ]:
#@title Instantiate YOLO+CLIP pipelines

# YOLOv5 + CLIP
yolo5clip = YoloClip(dataset.categories, yolo_ver="yolov5su", quiet=True, device=device)

# YOLOv8 + CLIP
yolo8clip = YoloClip(dataset.categories, yolo_ver="yolov8x", quiet=True, device=device)


In [ ]:
#@title Test the YOLOv5 + CLIP pipeline

yolo5clip(sample, sample.sentences[0], show_yolo=True, show=True, timeit=False)


In [ ]:
#@title Test the YOLOv8 + CLIP pipeline

yolo8clip(sample, sample.sentences[0], show_yolo=True, show=True, timeit=False)


### 2.2 Segmentation + CLIP

This approach is fundamentally based on the idea of segmenting the sample image into a set of regions and interpreting the similarity of each section to the prompt as a heatmap. An heatmap in this context is obtained as follows:

1. The sample image is segmented using a segmentation algorithm (e.g. SLIC, Watershed)
2. Each region is encoded using CLIP
3. The similarity between the prompt and each region is computed
4. Each "pixel" inside each region is assigned the similarity score of that region, producing a score heatmap

That being said, simply computing one single heatmap may not be enough to capture both larger and fine-grained image-text similarites. For such reasons, we compute different heatmaps using a different number of segments each time, and finally pooling all the heatmap together using the mean score of each pixel. This is not all though. At this point:

1. All pixels below a certain threshold are turned off (set to 0)
2. The heatmap is downsampled with a certain factor for lighter computations
3. The heatmap is normalized to the range [-1, 1]
4. The best bounding box is extracted from the heatmap, by considering all possible bounding boxes and selecting the one with the highest pixel sum

Note that:
- the threshold used to filter out pixels is taken as the value of a certain quantile `q` of the pixel values distribution
- the bbox search algorithm searches over all possible boxes, and that is why the heatmap is downsampled first
- the normalization also serves to give turned-off pixels negative values and discourage the algorithm from selecting areas containing many

Although being pretty flexible in terms of experimentation with hyperparameters, this model suffers from a few limitations, one of them being the slow inference time. This could be definitely improved in the future by defining another bounding box search algorithm, such as [...].


**Some implementative details***

The class implements a `__compute_hmap` method, which is responsible for computing the heatmap by passing a method and a number of segments to use it with. The method is used by `__call__` which performs the other steps described above and returns the metrics. The `_find_best_bbox` method is responsible for bbox searching, as decribed.

When calling the pipeline, the user can specify two additional parameters:

- `show_process=True` will show the resulting heatmap alongside its filtered and downsampled versions
- `show_masks=True` will show all *N* masks before pooling, with *N* being the number of segments used

Also, note that the hyperparameters specified in the class instantiation below are those which, experimentally, gave the best results.

In [ ]:
#@title Class definition for the Segmentation + CLIP pipeline

class ClipSeg(VisualGroundingPipeline):

    def __init__(self,
                 categories,
                 method,
                 n_segments,
                 clip_ver="ViT-L/14",
                 q=0.95,
                 d=16,
                 device="cpu",
                 quiet=False):

        VisualGroundingPipeline.__init__(self, categories, clip_ver, device, quiet)

        self.method = method
        self.n_segments = n_segments
        self.q = q
        self.d = d

        valid_methods = ["s", "w"]
        if self.method not in valid_methods:
            raise ValueError(f"Method `{method}` not supported. Supported methods are: {valid_methods}.")

        print("[INFO] Initializing ClipSeg pipeline")
        print(f"[INFO] Segmentation method: {method}")
        print(f"[INFO] Number of segments: {n_segments}")
        print(f"[INFO] Threshold q.tile for filtering: {q}")
        print(f"[INFO] Downsampling factor: {d}")
        print("")

    def _compute_hmap(self, img_sample, np_image, prompt, method, masks):

        # Make sure np_image is an image with shape (h, w, 3)
        if len(np_image.shape) > 3 or (len(np_image.shape) == 3 and np_image.shape[-1] != 3):
            np_image = np_image[:, :, 0]

        if len(np_image.shape) == 2:
            np_image = np.stack((np_image,) * 3, axis=-1)

        hmaps = list()

        prompt_enc = self._encode_text(prompt)

        for i, n in enumerate(masks):

            # Compute regions according to chosen method
            segments = None
            if method == "s":
                # SLIC segmentation algorithm ()
                segments = slic(np_image, n_segments=n, compactness=10, sigma=1)
            elif method == "w":
                # Watershed segmentation algorithm ()
                segments = watershed(sobel(rgb2gray(np_image)), markers=n, compactness=0.001)

            if segments is None:
                raise Exception("Segments are None. Is method different from 's' or 'w'? ")

            regions = regionprops(segments)

            if len(regions) == 1:
                # If the algo returned only 1 region, skip this iteration
                # (may happen, with low-segments masks)
                continue

            # Compute CLIP encodings for each region

            images_encs = list()

            regions = tqdm(regions, desc=f"[INFO] Computing CLIP masks", leave=False) if not self.quiet else regions

            for region in regions:
                rect = region.bbox
                rect = (rect[1], rect[0], rect[3], rect[2])

                sub_image = img_sample.img.crop(rect)
                image_enc = self._encode_img(sub_image)
                images_encs.append(image_enc)

            # Assign a score to each region according to prompt similarity (creating a heatmap)

            images_encs = torch.cat(images_encs, dim=0)
            scores = prompt_enc @ images_encs.T
            scores = scores.squeeze().cpu().numpy()
            heatmap = np.zeros((segments.shape[0], segments.shape[1]))

            for i in range(segments.shape[0]):
                for j in range(segments.shape[1]):
                    heatmap[i, j] = scores[segments[i, j] - 1]

            hmaps.append(heatmap)

        # Finally, return the pooled heatmap and the list of all heatmaps computed

        pmap = np.mean(np.array(hmaps), axis=0)

        return pmap, hmaps

    def _find_best_bbox(self, heatmap, lower_bound=-1.0, upper_bound=1.0):
        # Rescale the heatmap
        heatmap = MinMaxScaler(feature_range=(lower_bound, upper_bound)).fit_transform(heatmap)

        # Initialize the best score and best box
        best_score = float('-inf')
        best_box = None

        # Loop over all possible box sizes and positions
        for w in range(1, heatmap.shape[1] + 1):
            for h in range(1, heatmap.shape[0] + 1):
                for i in range(heatmap.shape[1] - w + 1):
                    for j in range(heatmap.shape[0] - h + 1):

                        # Get current sub-region
                        candidate = heatmap[j:j + h, i:i + w]

                        # Compute the score for this box
                        score = candidate.sum()

                        # Update the best score and best box if necessary
                        if score > best_score:
                            best_score = score
                            best_box = (i, j, w, h)

        best_box = [best_box[0], best_box[1], best_box[2] + best_box[0], best_box[3] + best_box[1]]

        return best_box

    def __call__(self, img_sample, prompt, show=False, show_process=False, show_masks=False, timeit=False):

        if timeit:
            start = time.time()

        """ Pipeline core """

        # Get sample image
        img = img_sample.img

        # Convert image to np array
        np_image = img_as_float(io.imread(img_sample.path))

        # Compute an heatmap of CLIP scores
        p_heatmap, heatmaps = self._compute_hmap(img_sample, np_image, prompt, self.method, self.n_segments)

        # Shut down pixels below a certain threshold
        ths = np.quantile(p_heatmap.flatten(), self.q)
        fp_heatmap = p_heatmap.copy()
        fp_heatmap[p_heatmap < ths] = ths

        # Downsample the heatmap by a factor d
        dfp_heatmap = downsample_map(fp_heatmap, self.d)

        # Find the best bounding box
        pred_bbox = self._find_best_bbox(dfp_heatmap, lower_bound=-0.75)

        if pred_bbox is None:
            return {"IoU": 0, "cosine": np.nan, "euclidean": np.nan, "dotproduct": np.nan, "grounding": np.nan}

        if self.d > 1:
            pred_bbox = [pred_bbox[0] * self.d + self.d // 2,
                         pred_bbox[1] * self.d + self.d // 2,
                         pred_bbox[2] * self.d - self.d // 2,
                         pred_bbox[3] * self.d - self.d // 2]

        # Use CLIP to encode the text prompt
        prompt_enc = self._encode_text(prompt).float()

        # Crop around the best bbox and encode
        pred_image = img.crop(pred_bbox)
        pred_image_enc = self._encode_img(pred_image)

        # Get ground truth bbox
        gt_bbox = img_sample.bbox

        """ Metrics computation """

        # Compute IoU
        iou = self._IoU(pred_bbox, gt_bbox)

        # Compute grounding accuracy
        grd_correct = self._grounding_accuracy(img_sample, pred_image_enc)

        # Compute distance metrics
        dotproduct = prompt_enc @ pred_image_enc.T  # dot product
        cosine_sim = cosine_similarity(prompt_enc, pred_image_enc)  # cosine similarity
        euclidean_dist = torch.cdist(prompt_enc, pred_image_enc, p=2).squeeze()  # euclidean distance

        """ Display results """

        # Show all masks, if requested
        if show_masks:
            fig, axes = plt.subplots(1, len(heatmaps), figsize=(20, 5))
            for i, heatmap in enumerate(heatmaps):

                for ax in axes.ravel():
                    ax.axis("off")

                axes[i].imshow(np_image, alpha=0.25)
                axes[i].imshow(heatmap, alpha=0.75)
                axes[i].set_title(f"#{i + 1}")

        # Show the mask processing pipeline, if requested
        if show_process:
            fig, axes = plt.subplots(1, 4, figsize=(20, 5))

            for ax in axes.ravel():
                ax.axis("off")

            axes[0].imshow(np_image)
            axes[0].set_title("original image")

            axes[1].imshow(np_image, alpha=0.25)
            axes[1].imshow(p_heatmap, alpha=0.75)
            axes[1].set_title("pooled heatmap")

            axes[2].imshow(np_image, alpha=0.25)
            axes[2].imshow(fp_heatmap, alpha=0.75)
            axes[2].set_title("filtered heatmap")

            axes[3].imshow(np_image, alpha=0.25)
            w, h = np_image.shape[1], np_image.shape[0]
            dfp_heatmap_ = cv2.resize(dfp_heatmap, (w, h), interpolation=cv2.INTER_NEAREST)
            axes[3].imshow(dfp_heatmap_, alpha=0.75)
            axes[3].set_title("dsampled heatmap")

        # Show the final prediction, if requested
        if show:
            methods = {"w": "Watershed", "s": "SLIC"}
            display_preds(img_sample.img, prompt, pred_bbox, img_sample.bbox,
                          f"{methods[self.method]} + CLIP ({self.clip_ver})")

        # Print execution time, if requested
        if timeit:
            end = time.time()
            print(f"[INFO] Time elapsed: {end - start:.2f}s")

        return {
            "IoU": float(iou),
            "cosine": float(cosine_sim),
            "euclidean": float(euclidean_dist),
            "dotproduct": float(dotproduct),
            "grounding": float(grd_correct),
        }


With that said, we can instantiate two object for the Segmentation+CLIP pipeline, which we will conveniently call after the segmentation method used.

In [ ]:
#@title Instantiate the segmentation + CLIP pipelines

# Watershed seg. + CLIP | pooling maps with 4, 8, 16, 32 segments | filtering below 0.75 q.tile
wshedclip = ClipSeg(dataset.categories, method="w", n_segments=(4, 8, 16, 32), q=0.75, quiet=False, device=device)

# SLIC seg. + CLIP | pooling maps with 4, 8, 16, 32 segments | filtering below 0.75 q.tile
slicnclip = ClipSeg(dataset.categories, method="s", n_segments=(4, 8, 16, 32), q=0.75, quiet=False, device=device)


In [ ]:
#@title Test the Watershed + CLIP pipeline

wshedclip(sample, sample.sentences[0], show_process=True, show_masks=True, show=True, timeit=False)


In [ ]:
#@title Test the SLIC + CLIP pipeline

slicnclip(sample, sample.sentences[0], show_process=True, show_masks=True, show=True, timeit=False)


### 2.3 SSD + CLIP

...

In [ ]:
#@title Class definition for the SSD + CLIP pipeline

class ClipSSD(VisualGroundingPipeline):

    def __new__(cls, *args, **kwargs):

        # Single Shot Detector (SSD) requires CUDA.
        # Check if the selected device if CUDA before instantiating the class.

        if kwargs["device"] != torch.device("cuda"):
            print("[ERROR] Single Shot Detector requires CUDA. Returning empty object.")
            print("")
            return VisualGroundingPipeline.__new__(VisualGroundingPipeline)
        else:
            return super(ClipSSD, cls).__new__(cls)

    def __init__(self,
                 categories,
                 confidence_t=0.5,
                 clip_ver="ViT-L/14",
                 device="cpu",
                 quiet=True):

        VisualGroundingPipeline.__init__(self, categories, clip_ver, device, quiet)

        self.confidence_t = confidence_t

        self.ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd')
        self.utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

        self.ssd_model.to(device)
        self.ssd_model.eval()

        print("[INFO] Initializing ClipSSD pipeline")
        print(f"[INFO] Confidence treshold: {confidence_t}")
        print("")

    def _propose(self, image_path, original_size):

        def _resize_bbox(bbox, in_size, out_size):
            """
            Resize bounding boxes according to image resize.

            Args:
                bbox: (np.ndarray) bounding boxes of (y_min, x_min, y_max, x_max)
                in_size: (tuple) the height and the width of the image before resized
                out_size: (tuple) The height and the width of the image after resized
            Returns:
                (np.ndarray) bounding boxes rescaled according to the given image shapes

            """
            bbox = bbox.copy()
            y_scale = float(out_size[0]) / in_size[0]
            x_scale = float(out_size[1]) / in_size[1]
            bbox[:, 0] = y_scale * bbox[:, 0]
            bbox[:, 2] = y_scale * bbox[:, 2]
            bbox[:, 1] = x_scale * bbox[:, 1]
            bbox[:, 3] = x_scale * bbox[:, 3]
            return bbox

        bboxes = []

        inputs = [self.utils.prepare_input(image_path)]
        tensor = self.utils.prepare_tensor(inputs)

        with torch.no_grad():
            detections_batch = self.ssd_model(tensor)

        results_per_input = self.utils.decode_results(detections_batch)
        best_results_per_input = [self.utils.pick_best(results, self.confidence_t) for results in results_per_input]

        bbox, _, _ = best_results_per_input[0]
        bbox *= 300
        bbox = _resize_bbox(bbox, (300, 300), original_size)
        bboxes.append(bbox)

        return np.float32(bboxes[0]).tolist()

    def __call__(self, img_sample, prompt, show=False, timeit=False):

        if timeit:
            start = time.time()

        """ Pipeline core """

        # Get sample image
        image_path = img_sample.path
        img = img_sample.img

        # Use SSD to propose relevant objects
        bboxes = self._propose(image_path, (img_sample.shape[1], img_sample.shape[2]))

        # Handle case where no object is proposed
        if len(bboxes) == 0:
            return {"IoU": 0, "cosine": np.nan, "euclidean": np.nan, "dotproduct": np.nan, "grounding": np.nan}

        # Use CLIP to encode each relevant object detected
        images_encs = list()
        for bbox in bboxes:
            sub_img = img.crop(bbox)
            with torch.no_grad():
                sub_img_enc = self._encode_img(sub_img)
            images_encs.append(sub_img_enc)
        images_encs = torch.cat(images_encs, dim=0)

        # Use CLIP to encode the text prompt
        prompt_enc = self._encode_text(prompt)

        # Find the best object according to cosine similarity
        c_sims = cosine_similarity(prompt_enc, images_encs).squeeze()
        best_idx = int(c_sims.argmax())

        # Get best bbox
        pred_bbox = bboxes[best_idx]

        # Use CLIP to encode the prompt
        prompt_enc = self._encode_text(prompt).float()

        # Crop around the best bbox and encode
        pred_image = img.crop(pred_bbox)
        pred_image_enc = self._encode_img(pred_image)

        # Get ground truth bbox
        gt_bbox = img_sample.bbox

        """ Metrics computation """

        # Compute IoU
        iou = self._IoU(pred_bbox, gt_bbox)

        # Compute grounding accuracy
        grd_correct = self._grounding_accuracy(img_sample, pred_image_enc)

        # Compute distance metrics
        dotproduct = prompt_enc @ pred_image_enc.T  # dot product
        cosine_sim = cosine_similarity(prompt_enc, pred_image_enc)  # cosine similarity
        euclidean_dist = torch.cdist(prompt_enc, pred_image_enc, p=2).squeeze()  # euclidean distance

        """ Display results """

        # Show the final prediction, if requested
        if show:
            display_preds(img, prompt, pred_bbox, gt_bbox, model_name="SSD+CLIP")

        # Print execution time, if requested
        if timeit:
            end = time.time()
            print(f"[INFO] Time elapsed: {end - start:.2f}s")

        return {
            "IoU": float(iou),
            "cosine": float(cosine_sim),
            "euclidean": float(euclidean_dist),
            "dotproduct": float(dotproduct),
            "grounding": float(grd_correct),
        }


With that said, we can instantiate and use an object for the SSD+CLIP pipeline.

In [ ]:
#@title Instantiate the SSD + CLIP pipeline

# SSD + CLIP | with 0.01 confidence
ssdnclip = ClipSSD(dataset.categories, confidence_t=0.01, device=device)


In [ ]:
#@title Test the SSD + CLIP pipeline

ssdnclip(sample, sample.sentences[0], show_ssd=True, show=True, timeit=False)


### 2.4 DETR + CLIP

...

In [ ]:
# Instantiate the DETR + CLIP pipeline

# DETR + CLIP
detrnclip = DetrClip(dataset.categories, quiet=True, device=device)


In [ ]:
#@title Test the DETR + CLIP pipeline

detrnclip(sample, sample.sentences[0], show_detr=True, show=True, timeit=False)


### 2.5 MDETR (SOTA)

...

In [ ]:
#@title Class definition for MDETR

class MDETRvg(VisualGroundingPipeline):

    def __init__(self,
                 categories,
                 clip_ver="RN101",
                 device="cpu",
                 quiet=True):

        VisualGroundingPipeline.__init__(self, categories, clip_ver, device, quiet)

        cpt_url = "https://pytorch.s3.amazonaws.com/models/multimodal/mdetr/pretrained_resnet101_checkpoint.pth"

        self.MDETR = mdetr_for_phrase_grounding()
        self.MDETR.load_state_dict(torch.hub.load_state_dict_from_url(cpt_url)["model_ema"])
        self.RoBERTa = RobertaTokenizerFast.from_pretrained("roberta-base")
        self.img_preproc = T.Compose([
            T.Resize(800),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

        print("[INFO] Initializing MDETR pipeline")
        print("")

    @staticmethod
    def rescale_boxes(boxes, size):
        w, h = size
        b = box_convert(boxes, "cxcywh", "xyxy")
        b = b * torch.tensor([w, h, w, h], dtype=torch.float32)
        return b

    def __call__(self, img_sample, prompt, show=True, timeit=False):

        if timeit:
            start = time.time()

        """ Pipeline core """

        # Get sample image
        img = Image.open(img_sample.path)

        # Make sure image has shape (h, w, 3)
        np_image = np.array(img)
        if len(np_image.shape) > 3 or (len(np_image.shape) == 3 and np_image.shape[-1] != 3):
            np_image = np_image[:, :, 0]
        if len(np_image.shape) == 2:
            np_image = np.stack((np_image,) * 3, axis=-1)
        img = Image.fromarray(np_image)

        # Encode the prompt with RoBERTa
        enc_text = self.RoBERTa.batch_encode_plus([prompt], padding="longest", return_tensors="pt")

        # Preprocess the image for MDETR
        img_transformed = self.img_preproc(img)

        # Run MDETR on image and prompt
        with torch.no_grad():
            out = self.MDETR([img_transformed], enc_text["input_ids"]).model_output

        # Parse MDETR results to get detections bboxes and probabilities
        probs = 1 - out.pred_logits.softmax(-1)[0, :, -1]
        boxes_scaled = self.rescale_boxes(out.pred_boxes[0, :], img.size)
        mdetr_results = pd.DataFrame(boxes_scaled.squeeze().numpy().reshape(-1, 4))
        mdetr_results.columns = ["xmin", "ymin", "xmax", "ymax"]
        mdetr_results["prob"] = probs.numpy()
        mdetr_results = mdetr_results.sort_values(by=['prob'], ascending=False)

        # Get best bbox
        pred_bbox = mdetr_results.iloc[0, :4].tolist()

        # Use CLIP to encode the prompt
        prompt_enc = self._encode_text(prompt)

        # Crop around the best bbox and encode
        pred_image = img.crop(pred_bbox)
        pred_image_enc = self._encode_img(pred_image)

        # Get ground truth bbox
        gt_bbox = img_sample.bbox

        """ Metrics computation """

        # Compute IoU
        iou = self._IoU(pred_bbox, gt_bbox)

        # Compute grounding accuracy
        grd_correct = self._grounding_accuracy(img_sample, pred_image_enc)

        # Compute distance metrics
        cosine_sim = cosine_similarity(prompt_enc, pred_image_enc)
        euclidean_dist = torch.cdist(prompt_enc, pred_image_enc, p=2).squeeze()
        dotproduct = prompt_enc @ pred_image_enc.T

        """ Display results """

        # Show the final prediction, if requested
        if show:
            display_preds(img, prompt, pred_bbox, gt_bbox, model_name="MDETR")

        # Print execution time, if requested
        if timeit:
            end = time.time()
            print(f"[INFO] Time elapsed: {end - start:.2f}s")

        return {
            "IoU": float(iou),
            "cosine": float(cosine_sim),
            "euclidean": float(euclidean_dist),
            "dotproduct": float(dotproduct),
            "grounding": float(grd_correct),
        }


With that said, we can instantiate and use an object for the MDETR pipeline.

In [ ]:
#@title Instantiate the MDETR pipeline

# MDETR for visual grounding
mdetr = MDETRvg(dataset.categories, quiet=True, device=device)


In [ ]:
#@title Test the MDETR pipeline

mdetr(sample, sample.sentences[0], show=True, timeit=False)


## 3. Testing and results

To test each visual grounding framework on the whole test split, a handy function is defined to run the pipeline on each sentence for each sample and compute the average metrics returned. The metrics are also printed in real time to monitor the progress of the testing process.

In [ ]:
#@title Function to test a given visual grounding pipeline on a given dataset

def visual_grounding_test(vg_pipeline, dataset, logging=False):
    scores = list()

    pbar = tqdm(dataset)

    for sample in pbar:

        sample = RefCOCOgSample(**sample)

        for sentence in sample.sentences:

            sc = vg_pipeline(sample, sentence, show=False)

            scores.append(sc)

            avg_metrics = list()

            for metric in scores[0].keys():
                avg_metric = np.mean([score[metric] for score in scores if score[metric] is not np.nan])
                avg_metric = f"{metric}: {avg_metric:.3f}"
                avg_metrics.append(avg_metric)

            pbar_desc = " | ".join(avg_metrics)

            if logging:
                pipeline_name = vg_pipeline.__class__.__name__.lower()
                datetime_tag = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

                with open(f"logs/{pipeline_name}_log_{datetime_tag}.txt", "a") as f:
                    f.write("[" + datetime_tag + "] " + pbar_desc + "\n")

            pbar.set_description(pbar_desc)


(You can use the dropdown menu to choose among one of the pipelines define above and test it right now. Still, notice the testing process can be pretty lenghty).

In [ ]:
#@title Test one of the pipeline defined

pipeline = yolo5clip  #@param ["yolo5clip", "yolo8clip", "wshedclip", "slicnclip", "detrnclip", "ssdnclip", "mdetr"]

visual_grounding_test(pipeline, test_ds)


### 3.1 Results discussion

...

### 3.2 Future directions

...

## 4 Interactive tool

Here you can use Colab interactive interface to play with the different implemented pipelines and customize their parameters.

#TODO